In [1]:
import duckdb
import os

file_path = r"C:\Users\KIIT\Desktop\synthetic_crm_data.csv"
con = duckdb.connect()

table_name = "crm_data"
con.execute(f"""
    CREATE TABLE {table_name} AS 
    SELECT * FROM read_csv_auto('{file_path}')
""")

schema_df = con.execute(f"PRAGMA table_info('{table_name}')").fetchdf()
schema_details = []
for _, row in schema_df.iterrows():
    col_name = row['name']
    col_type = row['type']
    # Get sample distinct values for this column (limited to 5)
    sample_values = con.execute(f"SELECT DISTINCT {col_name} FROM {table_name} LIMIT 5").fetchall()
    sample_str = ", ".join([str(val[0]) for val in sample_values if val[0] is not None])
    schema_details.append(f"{col_name} {col_type} - Sample values: {sample_str}")

head_df = con.execute(f"SELECT * FROM {table_name} LIMIT 5").fetchdf()
head_str = head_df.to_string(index=False)

In [2]:
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from openai import OpenAI
from langchain_core.output_parsers import StrOutputParser
import traceback


client = OpenAI(
    base_url='http://localhost:11434/v1',
    api_key='ollama'
)

user_query = [
    "Count of customers by status",
    "Count of customers by industry",
    "List of all churned customers",
    "Customers who haven't been contacted in the last 60 days"
]

system_template = """You are an expert SQL query generator specializing in DuckDB syntax. 
Generate precise, optimized SQL queries based on the schema provided.
Always follow these rules:
1. Only return the SQL query with no explanations or markdown formatting
2. For date calculations, always use CURRENT_DATE - INTERVAL n DAY
3. Do not use DATE('now'), NOW(), or MySQL/SQLite-style date functions
4. Use simple capital letters for main query aliases
5. Always reference the table as 'crm_data'
6. CAST timestamps/dates to the correct format when comparing
7. Never modify the schema
8. Be consistent in your query generation approach
"""

human_template = """Based on the schema and sample data below, write a SQL query that answers this question: {user_query}

SCHEMA DETAILS:
{schema_str}

SAMPLE DATA:
{head_str}

SQL QUERY:"""

chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template(human_template)
])



llm1 = ChatOpenAI(
    model="gemma3",
    openai_api_base="http://localhost:11434/v1",
    openai_api_key="ollama"
)

parser = StrOutputParser()
result = chat_prompt | llm1 | parser


def generate_query(user_queries):
    sql_queries = []
    for query in user_queries:
        sql = result.invoke({
            "schema_str": schema_details,
            "head_str": head_str,
            "user_query": query
        })
        # Remove any markdown formatting that might have been added
        sql = sql.replace("```sql", "").replace("```", "").strip()
        sql_queries.append(sql)
    return sql_queries


check_query_prompt = PromptTemplate.from_template("""
Check if the following SQL query is valid for the given schema. 
If any error is found, fix the query as per the schema and DuckDB requirements.
Return only the corrected SQL query, nothing else.

Schema:
{schema_str}

SQL Query:
{sql_query}
""")

checker_chain = check_query_prompt | llm1 | parser

def check(sql_queries_tuple):
    checked_queries = []
    for query in sql_queries_tuple:
        checked_sql = checker_chain.invoke({
            "schema_str": schema_details,
            "sql_query": query
        })
        checked_queries.append(checked_sql)
    return tuple(checked_queries)

def execute_query_with_error_capture(sql_query, con):
    try:
        result_df = con.execute(sql_query).fetchdf()
        return {"success": True, "result": result_df}
    except Exception as e:
        tb_str = traceback.format_exc()
        return {"success": False, "error_message": str(e), "traceback": tb_str}

error_fix_prompt = PromptTemplate.from_template("""
The following SQL query was executed in DuckDB and resulted in an error.
Please fix the query so it works, considering the schema and error message below.
Return only the corrected SQL query, in DuckDB SQL syntax.

Schema:
{schema_str}

SQL Query:
{sql_query}

Error Message:
{error_message}
""")

def auto_fix_and_execute(sql_query, con, schema_details, max_attempts=5):
    attempts = 0
    current_sql = sql_query
    while attempts < max_attempts:
        result = execute_query_with_error_capture(current_sql, con)
        if result["success"]:
            return result["result"]
        else:
            corrected_chain = error_fix_prompt | llm1 | parser
            current_sql = corrected_chain.invoke({
                "schema_str": schema_details,
                "sql_query": current_sql,
                "error_message": result["error_message"]
            })
        attempts += 1
    raise Exception("Failed to generate a correct SQL query after multiple attempts.")

def execute():
    results = generate_query(user_query)
    final = check(results)
    for idx, query in enumerate(final, start=1):
        print(f"\nQuery {idx}:")
        try:
            result_df = auto_fix_and_execute(query, con, schema_details)
            print(result_df)
        except Exception as e:
            print(f"Failed to execute corrected query after several attempts: {e}")

execute()


Query 1:
     Status  CustomerCount
0  Prospect             20
1  Customer             28
2      Lead             30
3   Churned             22

Query 2:
     Industry  CustomerCount
0     Finance             21
1  Technology             16
2      Retail             21
3  Healthcare             18
4   Education             24

Query 3:
                 Name
0         Robert Tran
1          Todd Kirby
2       Kathleen Wang
3        Alison Brown
4          Jorge Kidd
5       Brandon White
6   Elizabeth Proctor
7      Kaitlyn Monroe
8       Melissa Jones
9   Elizabeth Roberts
10    Ethan Hernandez
11     Valerie Larsen
12       George Lucas
13      Brandon Young
14  Phillip Gallagher
15        Jody Harvey
16  Christina Fischer
17     Amanda Webster
18     Julian Simmons
19        Jill Harper
20       Joshua Smith
21        Holly Gould

Query 4:
               Name                               Email  \
0        John Lewis  stevenblackburn@robbins-turner.com   
1     William Davis        

In [15]:
# sql_final